In [2]:
import geopandas as gpd
import shapely

In [3]:
edges = gpd.read_file('map-edges.gpkg')
nodes = gpd.read_file('map-nodes.gpkg')
truth = gpd.read_file('ground-truths.gpkg')
rawtr = gpd.read_file('raw-trajectory.gpkg')

In [4]:
nodes

,osmid,y,x,compressed_id,geometry
0,366372433,1.215633e+06,650430.259238,0,POINT (650430.259 1215633.107)
1,7251842686,1.215617e+06,650570.942426,1,POINT (650570.942 1215616.697)
2,7251774053,1.215637e+06,650325.182449,2,POINT (650325.182 1215636.828)
3,7258455914,1.215454e+06,650488.453585,3,POINT (650488.454 1215453.512)
4,366389789,1.215531e+06,651137.190174,4,POINT (651137.19 1215530.994)
...,...,...,...,...,...
146257,10991860089,1.149122e+06,708911.779484,146257,POINT (708911.779 1149122.168)
146258,12712705291,1.149992e+06,709157.651409,146258,POINT (709157.651 1149991.723)
146259,13266256212,1.148725e+06,709873.399449,146259,POINT (709873.399 1148724.597)
146260,13266256218,1.148763e+06,709716.362626,146260,POINT (709716.363 1148762.646)


In [5]:
edges

,index,u_compressed,v_compressed,oneway,maxspeed,geometry
0,0,0,1,False,40.0,"LINESTRING (650430.259 1215633.107, 650533.595..."
1,1,0,2,False,40.0,"LINESTRING (650430.259 1215633.107, 650334.831..."
2,2,0,3,False,40.0,"LINESTRING (650430.259 1215633.107, 650438.417..."
3,3,1,34,False,40.0,"LINESTRING (650570.942 1215616.697, 650583.625..."
4,4,1,0,False,40.0,"LINESTRING (650570.942 1215616.697, 650533.595..."
...,...,...,...,...,...,...
341358,341358,146259,145720,False,40.0,"LINESTRING (709873.399 1148724.597, 709898.407..."
341359,341359,146260,146146,False,40.0,"LINESTRING (709716.363 1148762.646, 709774.009..."
341360,341360,146260,146261,False,40.0,"LINESTRING (709716.363 1148762.646, 709710.415..."
341361,341361,146260,146145,False,40.0,"LINESTRING (709716.363 1148762.646, 709720.959..."


In [6]:
actual_start_points = shapely.get_point(edges.geometry.values, 0)
actual_end_points = shapely.get_point(edges.geometry.values, -1)

expected_u_node_geoms = nodes.loc[edges['u_compressed'], 'geometry'].values
expected_v_node_geoms = nodes.loc[edges['v_compressed'], 'geometry'].values

edges['is_aligned_with_u'] = shapely.equals(actual_start_points, expected_u_node_geoms)
edges['is_aligned_with_v'] = shapely.equals(actual_end_points, expected_v_node_geoms)

mismatch_u = edges[~edges['is_aligned_with_u']]
mismatch_v = edges[~edges['is_aligned_with_v']]

In [12]:
edges[edges['v_compressed'] == 12971]

,index,u_compressed,v_compressed,oneway,maxspeed,geometry,is_aligned_with_u,is_aligned_with_v
33364,33364,12968,12971,True,40.0,"LINESTRING (677852.747 1200197.166, 677858.641...",True,True


In [8]:
rawtr

,vehicle-number,routeID,varID,timestamps,geometry
0,51B09502,126,2,"[30, 60, 90, 120, 150, 180, 210, 240, 270, 300...","LINESTRING (685634.283 1187121.604, 685699.561..."
1,51B28745,92,1,"[30, 60, 90, 120, 150]","LINESTRING (670667.315 1155307.981, 671234.775..."
2,51B21569,92,1,"[30, 60, 90, 120, 150, 180, 210, 240, 270, 300...","LINESTRING (657408.219 1171983.261, 656895.66 ..."
3,51B16177,26,2,"[30, 60, 90, 120, 150, 180, 210, 240, 270, 300...","LINESTRING (676894.013 1187991, 676933.747 118..."
4,51B22443,65,129,"[30, 60, 90, 120, 150, 180, 210, 240, 270, 300...","LINESTRING (685441.801 1191677.967, 685480.885..."
...,...,...,...,...,...
2285,51B24025,114,227,"[30, 60, 90, 120, 150, 180, 210, 240, 270, 300...","LINESTRING (676156.788 1199534.166, 676005.473..."
2286,51B21962,130,261,"[30, 60, 90, 120, 150, 180, 210, 240, 270, 300...","LINESTRING (709710.155 1149850.675, 709949.896..."
2287,51B24205,114,227,"[30, 60, 90, 120, 150, 180, 210, 240, 270, 300...","LINESTRING (674422.908 1202084.116, 674176.011..."
2288,51B23925,114,227,"[30, 60, 90, 120, 150, 180, 210, 240, 270, 300...","LINESTRING (672085.535 1205418.073, 672079.824..."
